In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from scipy.stats import entropy
import numpy as np
import gc
import shapely
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

part = 9

bu = gpd.read_parquet(f'data/buurten_parts/buurten_c_{part}.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [2]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

In [3]:
bu.head(3)

,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,IND_WBI,H2O,POSTCODE,DEK_PERC,OAD,...,total_amenities_1,L0_leibovici_1,L1_leibovici_1,L0_altieri_1,L1_altieri_1,total_amenities_2,L0_leibovici_2,L1_leibovici_2,L0_altieri_2,L1_altieri_2
12549,BU19450003,Buitengebied Beek,WK194500,GM1945,Berg en Dal,1.0,NEE,6573,3.0,420.0,...,19.0,2.680872,3.612148,3.371272,4.642471,19.0,2.680872,3.612148,3.371272,4.642471
12550,BU19450100,Laag Berg en Dal,WK194501,GM1945,Berg en Dal,1.0,NEE,6571,3.0,548.0,...,7.0,1.318078,2.460252,1.667136,3.045053,7.0,1.318078,2.460252,1.667136,3.045053
12551,BU19450101,Hoog Berg en Dal,WK194501,GM1945,Berg en Dal,1.0,NEE,6571,5.0,327.0,...,11.0,3.492355,3.492355,4.330140,4.330140,10.0,2.992879,2.992879,3.829869,3.829869


In [4]:
for idx, row in tqdm(bu.iterrows(), total=bu.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        bu.at[idx, f'L0_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L0_category.values)
        bu.at[idx, f'L1_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L1_category.values)
        
        for cat in L0_cats:
            bu.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            bu.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

bu.to_parquet(f'data/buurten_parts/calculated/buurten_c_{part}.parquet') 

100%|██████████| 1394/1394 [18:45<00:00,  1.24it/s]
